<a href="https://colab.research.google.com/github/TejaishwaryaGagadam/style_transfer_ml/blob/master/Day8/Transfer_learning_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

now = datetime.datetime.now

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)


def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

In [0]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Instructions for updating:
Use tf.cast instead.
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 65s 2ms/step - loss: 0.1667 - acc: 0.9475 - val_loss: 0.0307 - val_acc: 0.9907
Epoch 2/5
30596/30596 [==============================] - 64s 2ms/step - loss: 0.0489 - acc: 0.9850 - val_loss: 0.0153 - val_acc: 0.9940
Epoch 3/5
30596/30596 [==============================] - 64s 2ms/step - loss: 0.0327 - acc: 0.9907 - val_loss: 0.0184 - val_acc: 0.9946
Epoch 4/5
30596/30596 [==============================] - 63s 2ms/step - loss: 0.0264 - acc: 0.9924 - val_loss: 0.0078 - val_acc: 0.9973
Epoch 5/5
30596/30596 [==============================] - 64s 2ms/step - loss: 0.0212 - acc: 0.9933 -

In [0]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

In [0]:
# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 24s 814us/step - loss: 0.2398 - acc: 0.9319 - val_loss: 0.0483 - val_acc: 0.9862
Epoch 2/5
29404/29404 [==============================] - 24s 808us/step - loss: 0.0761 - acc: 0.9767 - val_loss: 0.0338 - val_acc: 0.9883
Epoch 3/5
29404/29404 [==============================] - 24s 807us/step - loss: 0.0573 - acc: 0.9823 - val_loss: 0.0272 - val_acc: 0.9918
Epoch 4/5
29404/29404 [==============================] - 24s 811us/step - loss: 0.0493 - acc: 0.9845 - val_loss: 0.0262 - val_acc: 0.9905
Epoch 5/5
29404/29404 [==============================] - 24s 808us/step - loss: 0.0425 - acc: 0.9875 - val_loss: 0.0249 - val_acc: 0.9912
Training time: 0:01:59.323045
Test score: 0.024909029946863368
Test accuracy: 0.9911540834115528
